In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
import numpy as np
from IPython.display import display, Javascript  # Import Javascript class
from google.colab.output import eval_js
from base64 import b64decode
from google.colab.patches import cv2_imshow
import os
from PIL import Image

In [ ]:
# Load the face classifier and the emotion recognition model
face_classifier = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_frontalface_default.xml')
classifier = load_model('/content/drive/MyDrive/model.h5')
# feel free to change to your local path
mobilenet_classifier = load_model('/content/drive/MyDrive/CS6476/training_1/MobileNet-epoch20-batch64.keras')
pattlite_classifier = load_model('/content/drive/MyDrive/CS6476/training_1/patt-lite-retrained.keras')
emotion_labels_pattlite = ['Surprise', 'Fear', 'Disgust', 'Happy', 'Sad', 'Angry', 'Neutral']
emotion_labels_mobilenet = ['Surprise', 'Fear', 'Disgust', 'Happy', 'Sad', 'Angry', 'Neutral']
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

## Frame Preprocessing

In [ ]:
sample_video = '/content/drive/MyDrive/demo.mov'

In [ ]:
def extract_all_frames(video_path, step_size=1):
    video = cv2.VideoCapture(video_path)
    n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    results = []
    for i in range(0, n_frames, step_size):
        video.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = video.read()
        if ret:
            results.append(frame)
        else:
            break
    return np.array(results)


In [ ]:
video = cv2.VideoCapture(sample_video)
n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print(n_frames)

377


In [ ]:
frames = extract_all_frames(sample_video, step_size=10)

In [ ]:
def single_frame_fer(frame, fer_classifier, classifier_labels, color='rgb', output_size=(224,224)):
  faces = face_classifier.detectMultiScale(frame)
  # Process each face found
  for (x,y,w,h) in faces:
      cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
      if color =='rgb':
        roi_gray = frame[y:y+h, x:x+w, :]
      elif color =='gray':
        roi_gray = frame[y:y+h, x:x+w]
        roi_gray = cv2.cvtColor(roi_gray, cv2.COLOR_BGR2GRAY)
      roi_gray = cv2.resize(roi_gray, output_size, interpolation=cv2.INTER_AREA)
      if np.sum([roi_gray]) != 0:
          roi = roi_gray.astype('float') / 255.0
          roi = img_to_array(roi)
          roi = np.expand_dims(roi, axis=0)
          prediction = fer_classifier.predict(roi)[0]
          for i in range(len(prediction)):
            score = round(prediction[i], 3)
            labels = classifier_labels[i]
            text = str(labels) + ":  "+str(score)
            label_position = (x+w//3, h-20*i)
            cv2.putText(frame, text, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

          # Predict emotion

          label = classifier_labels[prediction.argmax()]
          label_position = (x-40, y - 10)  # Position above the rectangle for visibility
          cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

  # Display the resulting frame
  cv2_imshow(frame)

In [ ]:
for frame in frames:
  # frame = cv2.resize(frame, (224,224), interpolation=cv2.INTER_AREA)
  # res = pattlite_classifier.predict(frame)[0]
  faces = face_classifier.detectMultiScale(frame)
  # process each frame
  single_frame_fer(frame, pattlite_classifier, emotion_labels_pattlite)

Past scripts



In [ ]:


# Function to capture a photo
def take_photo(filename='photo.jpg', quality=0.8):
    js = '''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for Capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    '''
    display(Javascript(js))
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Capture and display a photo
try:
    filename = take_photo()  # Take a photo
    print('Saved to {}'.format(filename))

    # Display the captured image
    img = Image.open(filename)
    display(img)
except Exception as err:
    print(str(err))

# Load the captured image into OpenCV
frame = cv2.imread(filename)


emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
emotion_labels_mobilenet = ['Surprise', 'Fear', 'Disgust', 'Happy', 'Sad', 'Angry', 'Neutral']
emotion_labels_pattlite = ['Surprise', 'Fear', 'Disgust', 'Happy', 'Sad', 'Angry', 'Neutral']

# Convert frame to grayscale
# gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# faces = face_classifier.detectMultiScale(gray)

faces = face_classifier.detectMultiScale(frame)

# Process each face found
for (x,y,w,h) in faces:
    cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
    # roi_gray = gray[y:y+h, x:x+w]
    roi_gray = frame[y:y+h, x:x+w, :]
    # roi_gray = cv2.resize(roi_gray, (48,48), interpolation=cv2.INTER_AREA)
    roi_gray = cv2.resize(roi_gray, (224,224), interpolation=cv2.INTER_AREA)

    if np.sum([roi_gray]) != 0:
        roi = roi_gray.astype('float') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        prediction = pattlite_classifier.predict(roi)[0]
        for i in range(len(prediction)):
           score = round(prediction[i], 3)
           labels = emotion_labels_pattlite[i]
           text = str(labels) + ":  "+str(score)
           label_position = (x+w//3, h-20*i)
           cv2.putText(frame, text, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Predict emotion

        label = emotion_labels_pattlite[prediction.argmax()]
        label_position = (x-40, y - 10)  # Position above the rectangle for visibility
        cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

# Display the resulting frame
cv2_imshow(frame)


In [ ]:
capture = cv2.VideoCapture(2)

In [ ]:
# loop for real-time video capture
while True:
  ret, frame = capture.read()


  # Convert frame to grayscale
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  faces = face_classifier.detectMultiScale(gray)

  # faces = face_classifier.detectMultiScale(frame)

  # Process each face found
  for (x,y,w,h) in faces:
      cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
      roi_gray = gray[y:y+h, x:x+w]
      #  roi_gray = frame[y:y+h, x:x+w, :]
      roi_gray = cv2.resize(roi_gray, (48,48), interpolation=cv2.INTER_AREA)
      # roi_gray = cv2.resize(roi_gray, (224,224), interpolation=cv2.INTER_AREA)

      if np.sum([roi_gray]) != 0:
          roi = roi_gray.astype('float') / 255.0
          roi = img_to_array(roi)
          roi = np.expand_dims(roi, axis=0)
          prediction = classifier.predict(roi)[0]
          for i in range(len(prediction)):
            score = round(prediction[i], 3)
            labels = emotion_labels[i]
            text = str(labels) + ":  "+str(score)
            label_position = (x+w//3, h-20*i)
            cv2.putText(frame, text, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

          # Predict emotion
          label = emotion_labels[prediction.argmax()]
          label_position = (x-40, y - 10)  # Position above the rectangle for visibility
          cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
  cv2_imshow(frame)
  key = cv2.waitKey(1)
  if key == 27:
    break
capture.release()
cv2.destroyAllWindows()

